In [2]:
#!gsutil -m cp -r gs://neurips-adc-bucket/raw/train/100468857 ../data/raw/train
#!gsutil -m cp -r gs://neurips-adc-bucket/raw/train/1005054328 ../data/raw/train
#!gsutil cp gs://neurips-adc-bucket/raw/* ../data/raw/

In [3]:
from google.cloud import storage
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ibeam
import os
import subprocess
import glob
import shutil

In [4]:
storage_client = storage.Client()

In [5]:
blobs = storage_client.list_blobs(
    bucket_or_name="neurips-adc-bucket", match_glob="raw/**/*", delimiter="/"
)

In [6]:
[blob for blob in blobs.prefixes]

[]

In [7]:
def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    # Note: The call returns a response only when the iterator is consumed.
    for blob in blobs:
        print(blob.name)

In [8]:
def copy_to(element, dest: str):
    sufix = "/".join(element.split("/")[-2:])
    output = os.path.join(dest, sufix)
    shutil.copytree(src=element, dst=output)

In [10]:
with beam.Pipeline() as pipeline:
    outputs = (
        pipeline
        | "Create initial values" >> beam.Create(glob.glob("../data/raw/**/*"))
        | "Copy the folder" >> beam.Map(lambda x: copy_to(x, "../data/output"))
    )
    outputs | beam.Map(print)

None
None


Next setp appliquer la transformation de bout en bout dans le notebook.
quand ce sera bon on va voir comment intéger cela dans un ExampleGen

# LS FUNCTION

In [11]:
from typing import Union
from pathlib import Path

In [12]:
type(os.path.join(".", "data"))

str

In [13]:
def ls_gcp_bucket(args: list[str]) -> list[str]:
    """List providers, buckets, or objects

    Args:
        command (list[str]): args and options supproted by `gsutil ls`

    Returns:
        list[str]: Informations returned by `gsutil ls` command as list for each line.
    """
    command = ["gsutil", "ls"] + args
    result = subprocess.run(args=command, capture_output=True, text=True)
    if result.returncode != 0:
        # TODO: logging
        print(f"{result.stderr}")
    else:
        output = result.stdout.splitlines()
        return output

In [14]:
os.listdir()

['02-host-update-calibrating-and-binning-astronomical-data.ipynb',
 '03-host-starter-solution.ipynb',
 '01-cay-understanding-the-data.ipynb',
 '.gitkeep',
 '04-cay-test-apache-beam-tfx.ipynb']

In [15]:
def ls_filesystem(
    path: Union[str, Path], files_only: bool = False, folders_only: bool = False
) -> list[str]:
    """List files in path.

    Args:
        path (Union[str, Path]): The path whose objects to list.
        files_only (bool, optional): Whether to return files only. Defaults to False.
        folders_only (bool, optional): Whether to return folders only. Defaults to False.

    Returns:
        list[str]: list of element in path.
    """
    if files_only and folders_only:
        folders_only = False
    ls = [os.path.join(path, item) for item in os.listdir(path)]
    if files_only:
        files = []
        for item in ls:
            if os.path.isfile(item):
                files.append(item)
        ls = files
    if folders_only:
        folders = []
        for item in ls:
            if os.path.isdir(item):
                folders.append(item)
        ls = folders
    return ls

In [20]:
import pandas as pd
import numpy as np

In [17]:
class ProcessingDoFn(beam.DoFn):
    def __init__(self, axis_info, train_adc, test_adc):
        self.axis = axis_info
        self.adc = train_adc
        self.tadc = test_adc

    def process(self, element):
        return [self.axis, self.adc, self.tadc, element]

In [ ]:
class ADCRevertFn:
    def process(self, element):
        airs_files = {}
        fgs_files = {}
        for item in os.listdir(element):
            if "airs" in item.lower():
                if "signal" in item.lower():
                    airs_files["signal"] = pd.read_parquet(path=item)
                if os.path.isdir(item):
                    for file in os.listdir(item):
                        if "dark" in file:
                            airs_files["dark"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
                        if "dead" in file:
                            airs_files["dead"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
                        if "flat" in file:
                            airs_files["flat"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
                        if "read" in file:
                            airs_files["read"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
            if "fgs" in item.lower():
                if "signal" in item.lower():
                    fgs_files["signal"] = pd.read_parquet(path=item)
                if os.path.isdir(item):
                    for file in os.listdir(item):
                        if "dark" in file:
                            fgs_files["dark"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
                        if "dead" in file:
                            fgs_files["dead"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
                        if "flat" in file:
                            fgs_files["flat"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
                        if "read" in file:
                            fgs_files["read"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
        airs_files["signal"] = (
            fgs_files["signal"]
            .astype(np.float64)
            .reshape((fgs_files["signal"].shape[0], 32, 356))
        )
        airs_files["signal"]
        files = os.listdir(element)
        signal = signal.astype(np.float64)
        signal /= gain
        signal += offset
        return signal

In [25]:
l = ls_gcp_bucket(["gs://neurips-adc-bucket/**"])

In [34]:
l

['gs://neurips-adc-bucket/raw/',
 'gs://neurips-adc-bucket/raw/axis_info.parquet',
 'gs://neurips-adc-bucket/raw/sample_submission.csv',
 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/dark.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/dead.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/flat.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/linear_corr.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/read.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_signal.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/dark.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/dead.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/flat.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/linear_corr.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/rea

In [39]:
"gs://neurips-adc-bucket/raw/train/1468955155/FGS1_signal.parquet".split("/")[5]

'1468955155'

In [45]:
def split_key_value(path: str):
    l = path.split("/")
    if l[4] != "" and len(l) == 5:
        return (l[3], "/".join(l))
    if len(l) > 5:
        return (l[6], "/".join(l))

In [28]:
"gs://neurips-adc-bucket/raw/".split("/")

['gs:', '', 'neurips-adc-bucket', 'raw', '']

In [19]:
with beam.Pipeline() as pipeline:
    train_adc = pd.read_csv("../data/raw/train_adc_info.csv")
    test_adc = pd.read_csv("../data/raw/test_adc_info.csv")
    axis = pd.read_parquet("../data/raw/axis_info.parquet")
    outputs = (
        pipeline
        | "Create initial values"
        >> beam.Create(ls_gcp_bucket(["gs://neurips-adc-bucket/**"]))
        | "Ls filesystem" >> beam.Map(lambda x: copy_to(x, "../data/output"))
    )
    outputs | beam.Map(print)

        AIRS-CH0-axis0-h  AIRS-CH0-axis2-um  AIRS-CH0-integration_time  \
0               0.000028           4.078463                        0.1   
1               0.000056           4.074023                        4.5   
2               0.001361           4.069568                        0.1   
3               0.001389           4.065100                        4.5   
4               0.002694           4.060618                        0.1   
...                  ...                ...                        ...   
134995               NaN                NaN                        NaN   
134996               NaN                NaN                        NaN   
134997               NaN                NaN                        NaN   
134998               NaN                NaN                        NaN   
134999               NaN                NaN                        NaN   

        FGS1-axis0-h  
0           0.000028  
1           0.000056  
2           0.000139  
3           0.00016

In [47]:
class ProcessingDoFn(beam.DoFn):
    def __init__(self, axis_info, train_adc, test_adc):
        self.axis = axis_info
        self.adc = train_adc
        self.tadc = test_adc

    def process(self, element):
        return [self.axis, self.adc, self.tadc, element]

In [50]:
class ProcessingDoFn(beam.DoFn):
    def process(self, element):
        return [element[1]]

In [51]:
with beam.Pipeline() as pipeline:
    train_adc = pd.read_csv("../data/raw/train_adc_info.csv")
    test_adc = pd.read_csv("../data/raw/test_adc_info.csv")
    axis = pd.read_parquet("../data/raw/axis_info.parquet")
    outputs = (
        pipeline
        | "Create initial values"
        >> beam.Create(ls_filesystem(path="../data/raw/train"))
        | "Ls filesystem" >> beam.Map(lambda x: split_key_value(path=x))
        | "Group" >> beam.GroupByKey()
        | "Par Do" >> beam.ParDo(ProcessingDoFn())
    )
    outputs | beam.Map(print)

['../data/raw/train/100468857', '../data/raw/train/1005054328']
